# Understanding the Kugelman et al 2019 Code
## Data for this Jupyter notebook
These files need to be in the same directory as this jupyter notebook:

1. LI = Lab images (Aconitums.)
2. KEX = (Kugelman et al 2009) example_data.hdf5

## Semantic Model
The code in https://github.com/jakugel/oct-choroid-seg will be analyzed and modified to read in area segmentations instead of boundaries, and to decide whether to use hdf5 format for the images or read the images directly from a directory.

The boundaries are the ILM, RPE and CSI. For the semantic segmentation these 4 regions (areas) are needed:

1. Vitreous (top of the image to ILM)
2. Retinal (ILM to RPE)
3. Choroid (RPE to CHR)
4. Sclera (CHR to bottom of the image)

## Loading Area Labels
The semantic model requires image area segmentations, but the functions `load_training_data` and `load_validation_data` in `train_script_semantic_general.py` are written for segmentation files that contain boundaries.

These boundaries are then converted to areas or “masks” in function `create_all_area_masks(images, segs)`.

If the input training labels are already area labels, the function `create_all_area_masks` does not need to convert boundaries to areas. It should be enough to set `mask = segs`.

## Dependencies
See ReadMe.md

# Understanding the Images
## Directory structure and HDF5 structure
LI = Lab images
KEX = (Kugelman et al 2009) example_data.hdf5

The KEX file can be read with hdf52images.py and has this structure:

test_images
Dims: (3, 1536, 496, 1)
test_segs
Dims: (3, 3, 1536)
train_images
Dims: (3, 1536, 496, 1)
train_segs
Dims: (3, 3, 1536)
val_images
Dims: (3, 1536, 496, 1)
val_segs
Dims: (3, 3, 1536)

The 4 dimensions (called 'shape' in hdf5 format) of the images are :

num_images = images.shape[0]
image_width = images.shape[1]
image_height = images.shape[2]
num_channels = images.shape[3]

The remlmaterials directory uses the same structure, and the LI images are read from the directory not an hdf5 file.

The format is:

val_images
3 tif files: (496, 1536)
val_segs
3 png files: (496, 1536)
... same for test and train datasets.

Differences with example_data.hdf5
KEX images need to be rotated -90 or + 270 degrees to be in the same angle as RGLI
LI use RGB colors and need to be converted to greyscale
LI image format is hdf5, KEX is TIF for the images and png for the segmentations/labels
KEX uses boundaries as labels, RGI uses areas
LI has scanning instrument label on the images
KEX accounts for RGB or greyscale with the 4th dimension of "channels"
Code Incompatibilities
The (Kugelman et al 2009) code makes extensive use of the hdf5 format. Some functions, such as ImageDatabase in image_database.py expect hdf5 datasets as arguments.

The code below converts a numpy array to an hdf5 dataset, but requires the creation of an hdf5 file.

In [ ]:
import numpy as np
import h5py
from PIL import Image, ImageOps

# load image as is
image = np.array(Image.open('Aconitums004.tif'))
h5f = h5py.File('data.h5', 'w')
try:
    h5imgarray=h5f.create_dataset('train_images', data=image)
    print("\nLI")
    # The code below outputs the same dimenions as print("Dims: " + str(dset.shape))
    # But clarifies the hdf5 dataset structure
    for i in range(h5imgarray.ndim):
        print(h5imgarray.shape[i])
finally:
    h5f.close()

#load KEX images
h5f = h5py.File('example_data.hdf5', 'r')
try:
    h5=h5f['train_images'][:]
    print("\nKEX")
    for i in range(h5.ndim):
        print(h5.shape[i])
finally:    
    h5f.close()

#Modify LI to the same format as KEX
train_images = []
# Convert to greyscale and rotat
train_images.append(np.rot90(np.array(ImageOps.grayscale(Image.open('Aconitums002.tif'))),3))
train_images.append(np.rot90(np.array(ImageOps.grayscale(Image.open('Aconitums003.tif'))),3))
train_images.append(np.rot90(np.array(ImageOps.grayscale(Image.open('Aconitums004.tif'))),3))
# Add the 4th axis with dimension 1 
train_images=np.asarray(train_images)
train_images=train_images.reshape(3, 1536,496,1)
h5f = h5py.File('data.h5', 'w')
try:
    h5imgarray=h5f.create_dataset('train_images', data=train_images)
    print("\nLI Reformatted")
    for i in range(h5imgarray.ndim):
        print(h5imgarray.shape[i])

finally:
    h5f.close()

# Custom Code Modifications
## Dependencies
`environment.yml`

## Processing images
`create_all_area_masks(images, segs)` calls `create_area_mask(image, segs)` for all training or evaluation image and label pairs. In our dataset, images are TIF files, and segs are PNG files.

The images, segs parameters of `create_all_area_masks` are HDF5 datasets, not regular arrays. The return value is an np.array.

We modify `create_area_mask(image, segs)` to set `mask = segs` **if the image and segs dimensions and dtype are the same and dtype is uint8.**

*Warning: Matplolib reads png files with function png.read_png_float by default which makes it seem integer png files are of type float32. Use PIL instead.*

The code below uses PIL to inspect the images:

In [ ]:
import numpy as np
from matplotlib import pyplot
from PIL import Image

def create_area_mask(image, segs):
    mk = np.array([])    
    #verify image & segs have the same integer dimensions
    if image.shape == segs.shape and segs.dtype == image.dtype and np.issubdtype(np.uint8,segs.dtype) :
        mk = segs
    
    return mk

# load image as pixel array
image = np.array(Image.open('Aconitums004.tif'))

# Verify image is correct: summarize shape of the pixel array
print(image.dtype)
print(image.shape)

# Verify image is correct: display the array of pixels as an image

pyplot.imshow(image)
pyplot.show()

# load image as pixel array
# Be aware that matplotlib uses by default png.read_png_float for png images, 
# use PIL instead:
segs = np.array(Image.open('Aconitums004.png'))



mask = create_area_mask(image, segs)
if mask.all():
  print("Mask is empty")
else:
    # Verify mask is correct: summarize shape of the pixel array
    print(mask.dtype)
    print(mask.shape)

    # Verify mask is correct: display the array of pixels as an image

    pyplot.imshow(mask)
    pyplot.show()